In [2]:
import pandas as pd
import requests
import re
# get surburb list
post = []
postcode = pd.read_excel("../data/suburb.xltx")
surburb = postcode['Unnamed: 1']
surburb = surburb.dropna()
surburb = surburb.drop_duplicates().reset_index().drop('index',axis = 1).rename({'Unnamed: 1':'surburb'}, axis = 1)
surburb

,surburb
0,Albert Park-Middle Park-West St Kilda
1,Armadale
2,Carlton North
3,Carlton-Parkville
4,CBD-St Kilda Rd
...,...
142,Traralgon
143,Wanagaratta
144,Warragul
145,Warrnambool


In [4]:
# get postcode and surburb corresponding list
suburb_postcode_df = pd.read_csv('../data/cleaned_complete.csv')[['postcode','surburb']].drop_duplicates()
d = dict()
for index, values in suburb_postcode_df.iterrows():
    d[values['postcode']] = values['surburb']

In [5]:
missing = pd.read_csv('../data/curated/missing_houses.csv')
missing = missing.drop(['Unnamed: 0', 'count'], axis = 1)
suburb_list = []
for index, value in missing.iterrows():
    suburb = d[value['postcode']]
    suburb_list.append(suburb)

missing['suburb'] = suburb_list
missing

,postcode,suburb
0,3747,beechworth
1,3744,wandiligong
2,3644,cobram
3,3792,the patch
4,3779,marysville
...,...,...
195,3505,merbein south
196,3616,tatura
197,3842,churchill
198,3916,point leo


In [6]:
surburb

,surburb
0,Albert Park-Middle Park-West St Kilda
1,Armadale
2,Carlton North
3,Carlton-Parkville
4,CBD-St Kilda Rd
...,...
142,Traralgon
143,Wanagaratta
144,Warragul
145,Warrnambool


In [ ]:
# get suburb id for each suburb
result_list = []
for item in surburb['surburb']:
    suburbs = item.split("-")
    for suburb in suburb_list:
        suburb_replaced = suburb.replace(" ", "%20")
        print("request suburb id for: ", suburb_replaced)
        url = f"https://api.domain.com.au/v1/addressLocators?searchLevel=Suburb&suburb={suburb_replaced}&state=VIC"
        headers = {"accept": "application/json", "X-Api-Key": "key_a68db2bad8d7006e8aa1a5326d638dd8"}
        response = requests.get(url, headers=headers)
        if response.status_code != 200:
            print("request failed", response.reason)
            continue
        data = response.json()
        suburb_id_list = [suburb_info["ids"][0]["id"] for suburb_info in data]
        for suburb_id in suburb_id_list:
            result_list.append([suburb, suburb_id])


In [62]:
pd.DataFrame(result_list, columns= ['surburb', 'suburb_id']).drop_duplicates().to_csv("../data/get_house_data/suburb_id.csv", index=False)

In [9]:
# read suburb id file
df2 = pd.read_csv("../data/suburb_id.csv")
df2['suburb'] = df2['suburb'].apply(lambda x:x.lower())
df2 

,suburb,suburb_id
0,albert park,797
1,middle park,26507
2,armadale,1587
3,carlton north,8247
4,carlton,8217
...,...,...
205,torquay,40507
206,traralgon,40687
207,warragul,42517
208,warrnambool,42697


In [ ]:

headers = {"accept": "application/json", "X-Api-Key": "key_3672a55119b75bf70117593dbf4698ba"}
# get agent lists for each suburb
agent_list = []
for index, item in df2[:79].iterrows():
    suburb_id = item["suburb_id"]
    print(suburb_id)
    url = f"https://api.domain.com.au/v1/agencies?q=suburbId%3A{suburb_id}&pageNumber=1&pageSize=10000"
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        print("request failed", response.reason)
        continue
    data = response.json()
    if len(data) == 0:
        break
    if isinstance(data, dict):
        agent_list.extend([data])
    else:
        agent_list.extend(data)
df = pd.DataFrame(agent_list)
df.to_csv('../data/agent_list1.csv')

In [ ]:
for index, item in df2[79:].iterrows():
    suburb_id = item["suburb_id"]
    print(suburb_id)
    url = f"https://api.domain.com.au/v1/agencies?q=suburbId%3A{suburb_id}&pageNumber=1&pageSize=10000"
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        print("request failed", response.reason)
        continue
    data = response.json()
    if len(data) == 0:
        break
    if isinstance(data, dict):
        agent_list.extend([data])
    else:
        agent_list.extend(data)
df = pd.DataFrame(agent_list)
df.to_csv('../data/agent_list2.csv')

In [11]:
def to_int(text):
    if len(text.split(' ')) == 1:
        return int(text)
    else:
        return 0

In [ ]:
df1 = pd.read_csv('../data/get_house_data/agent_list.csv',on_bad_lines='skip')
# only keep agent that has rent houses
df1 = df1[~(df1['numberForRent'].isnull())]
df1['numberForRent'] = df1['numberForRent'].apply(to_int)
df2 = pd.read_csv('../data/get_house_data/agent_list2.csv',on_bad_lines='skip')
agent_info = pd.concat([df1, df2]).reset_index().drop(['index',	'Unnamed: 0'], axis = 1)
agent_info = agent_info[agent_info['numberForRent'] > 0]
agent_info_id = agent_info['id']
agent_info_id = agent_info_id.drop_duplicates()
agent_info_id.to_csv('../data/get_house_data/agent_info_id.csv')

In [19]:
agent_info_id = pd.read_csv('../data/get_house_data/agent_info_id.csv')
agent_info_id = agent_info_id.drop('Unnamed: 0', axis = 1)
agent_info_id

,id
0,2783
1,17566
2,34735
3,4307
4,29954
...,...
1136,28341
1137,19058
1138,6983
1139,17005


In [ ]:
# get house data according to agent id
# Need to get house data multiple times
house_list = []
headers = {'accept': 'application/json','X-Api-key':'key_2064b0fe29a00f6a734ff057907cb717'}
for i in agent_info_id.id:
    print(i)
    url = f'https://api.domain.com.au/v1/agencies/{i}/listings?listingStatusFilter=liveAndArchived&pageNumber=1&pageSize=100000'
    response = requests.get(url,headers = headers)
    data = response.json()
    if len(data) == 0:
        continue
    if isinstance(data, dict):
        house_list.extend([data])
    else:
        house_list.extend(data)

In [ ]:
df = pd.DataFrame(house_list)
df.to_csv('../data/raw/house_list.csv')

In [20]:
house_data = pd.read_csv('../data/raw/house_list.csv')

In [21]:
# only keep interested columns
house_df = house_data[['propertyTypes','objective','addressParts','advertiserIdentifiers','bathrooms','bedrooms', 'carspaces','dateUpdated', 'dateListed','geoLocation','priceDetails','landAreaSqm','dateAvailable']]
# only keep houses that are rent
house_df = house_df[house_df['objective'] == 'rent']
house_df = house_df.reset_index().drop('index', axis = 1)
house_df.to_csv('../data/raw/house_df.csv')

In [22]:
# extract suburb from address
suburb_df = []
for i in range(len(house_df['addressParts'])):
    s = re.findall(r'\'suburb\': +\'+[A-Za-z\' \']+',house_df['addressParts'][i])
    if(len(s) == 0):
        suburb_df.append(' ')
    else:
        s = s[0].split(':')[1][2:-1].lower()
        suburb_df.append(s)

In [24]:
# extract postcode from address
postcode_df = []
for i in range(len(house_df['addressParts'])):
    postcode = re.findall(r'postcode\': \'[0-9]+',house_df['addressParts'][i])
    if(len(postcode) == 0):
        postcode_df.append(0)
    else:
        postcode = postcode[0].split(':')[1][2:]
        postcode_df.append(int(postcode))

In [87]:
# add surburb and postcode into the dataframe
suburb_df = pd.DataFrame(suburb_df)
postcode_df = pd.DataFrame(postcode_df)
house_df['postcode'] = postcode_df
house_df['surburb'] = suburb_df

In [88]:
# consider the dateUpdated as the year and month of house that is rented 
year_df = []
month_df = []
for i in range(len(house_df['addressParts'])):
    date = house_df['dateUpdated'][i]
    if(len(date) == 0):
        year_df.append(0)
        month_df.append(0)
    else:
        year = date[:4]
        month = date[5:7]
        year_df.append(int(year))
        month_df.append(int(month))


In [89]:
year_df = pd.DataFrame(year_df)
month_df = pd.DataFrame(month_df)
house_df['year'] = year_df
house_df['month'] = month_df

In [90]:
# only keep house that is rented after year2018
house_df = house_df[house_df['year']> 2018]
house_df

,propertyTypes,objective,addressParts,advertiserIdentifiers,bathrooms,bedrooms,carspaces,dateUpdated,dateListed,geoLocation,priceDetails,landAreaSqm,dateAvailable,postcode,surburb,year,month
105,['house'],rent,"{'stateAbbreviation': 'vic', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 1...",1.0,3.0,1.0,2019-02-12T04:53:24.147Z,2014-06-04T04:40:45Z,"{'latitude': -36.3567799, 'longitude': 146.688...","{'canDisplayPrice': False, 'displayPrice': '$3...",NaN,NaN,3747,beechworth,2019,2
110,['house'],rent,"{'stateAbbreviation': 'vic', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 1...",1.0,3.0,0.0,2020-03-27T03:59:01.72Z,2020-02-11T23:09:18Z,"{'latitude': -36.3672451, 'longitude': 146.681...","{'price': 340.0, 'pricePrefix': '$340 per week...",NaN,2020-03-20,3747,beechworth,2020,3
111,['house'],rent,"{'stateAbbreviation': 'vic', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 1...",1.0,4.0,1.0,2019-11-24T23:57:24.163Z,2019-09-16T00:14:18Z,"{'latitude': -36.3571552, 'longitude': 146.687...","{'canDisplayPrice': False, 'displayPrice': '$3...",NaN,2019-10-18,3747,beechworth,2019,11
112,['house'],rent,"{'stateAbbreviation': 'vic', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 1...",1.0,3.0,1.0,2020-05-05T05:32:13.28Z,2020-04-14T04:46:54Z,"{'latitude': -36.3545598, 'longitude': 146.689...","{'canDisplayPrice': False, 'displayPrice': '$3...",NaN,2020-05-08,3747,beechworth,2020,5
114,['house'],rent,"{'stateAbbreviation': 'vic', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 1...",1.0,2.0,0.0,2019-11-25T04:02:27.697Z,2019-11-13T00:16:40Z,"{'latitude': -36.3560633, 'longitude': 146.689...","{'canDisplayPrice': False, 'displayPrice': '$2...",NaN,2019-12-19,3747,beechworth,2019,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59058,['house'],rent,"{'stateAbbreviation': 'vic', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 2...",2.0,4.0,4.0,2022-09-05T23:17:47.523Z,2022-08-05T03:33:29Z,"{'latitude': -37.2844387, 'longitude': 144.728...","{'canDisplayPrice': False, 'displayPrice': 'UN...",NaN,2022-08-11,3435,lancefield,2022,9
59059,['house'],rent,"{'stateAbbreviation': 'vic', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 2...",2.0,4.0,2.0,2022-09-27T23:34:22.997Z,2022-08-21T23:50:38Z,"{'latitude': -37.2844387, 'longitude': 144.728...","{'canDisplayPrice': False, 'displayPrice': 'UN...",567.91,2022-08-22,3435,lancefield,2022,9
59060,['house'],rent,"{'stateAbbreviation': 'vic', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 2...",1.0,2.0,0.0,2022-09-27T04:46:42.247Z,2022-09-25T23:29:15Z,"{'latitude': -37.3460627, 'longitude': 144.741...","{'canDisplayPrice': False, 'displayPrice': '$4...",NaN,2022-10-28,3434,romsey,2022,9
59061,['house'],rent,"{'stateAbbreviation': 'vic', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 2...",2.0,4.0,4.0,2022-10-03T23:30:24.74Z,2022-09-26T03:48:38Z,"{'latitude': -37.3626928, 'longitude': 144.747...","{'canDisplayPrice': False, 'displayPrice': 'UN...",813.00,2022-10-03,3434,romsey,2022,10


In [91]:
house_df = house_df.reset_index()
house_df['year'].value_counts()

2019    2383
2020    2215
2021    2112
2022    1893
Name: year, dtype: int64

In [92]:
# get the bond, price of houses
def get_bond(text):
    text_dict = eval(text)
    return text_dict.get('bond')
def get_price(text):
    text_dict = eval(text)
    if text_dict.get('price'):
        return text_dict.get('price')
    else:
        return text_dict.get('displayPrice')

def to_lower(text):
    if isinstance(text,float):
        return text
    text = text.lower()
    return text

In [93]:
house_df['price'] = house_df['priceDetails'].apply(get_price)
house_df['bond'] = house_df['priceDetails'].apply(get_bond)
house_df['price'] = house_df['price'].apply(to_lower)

In [96]:
# get the weekly rent of each house
def get_weekly_price(text):
    if isinstance(text,float):
        return text
    money = re.findall(r'\$\d*[,.]?\d+[,.]?\d+',text)
    if len(money) == 0:
        return None
    elif len(money) == 1:
        money = money[0].replace(',','')[1:]
        if money.count('.') == 2:
            return None
        else:
            return float(money)
    else:
        if('per week' in text):
            num_text = re.findall(r'\$\d*[,.]?\d+[,.]?\d+ per week',text)
            if (len(num_text)== 1):
                return float(num_text[0].split(' ')[0][1:].replace(',',''))
            else:
                total = 0
                for i in num_text:
                    i = i.split(' ')[0][1:].replace(',','')
                    total += float(i)
                return total/len(num_text)
        elif('pw' in text):
            text1 = re.findall(r'\$\d*[,.]?\d+[,.]?\d+ pw',text)
            if(len(text1) == 1):
                return float(text1[0].split(' ')[0][1:].replace(',',''))
            elif(len(text1) > 1):
                total = 0
                for i in text1:
                    i = i.split(' ')[0][1:].replace(',','')
                    total += float(i)
                return total/len(text1)
            else:
                text2 = re.findall(r'\$\d*[,.]?\d+[,.]?\d+pw',text)
                if(len(text2)==1):
                    if(text2[0][0] != '$'):
                        return float(text2[0][:-2].replace(',',''))
                    else:
                        return float(text2[0][1:-2].replace(',',''))
                elif(len(text2) > 1):
                    total = 0
                    for i in text2:
                        i = i.split(' ')[0][1:-2].replace(',','')
                        total += float(i)
                    return total/len(text2)
        elif('p/w' in text):
            text3 = re.findall(r'\$\d*[,.]?\d+[,.]?\d+ p/w',text)
            if(len(text3)==1):
                text3 = text3[0].split(' ')[0][1:].replace(',','')
                return float(text3)
            elif(len(text3)>1):
                total = 0
                for i in text3:
                    i = i.split(' ')[0][1:-2].replace(',','')
                    total += float(i)
                return total/len(text3)
            else:
                text4 = re.findall(r'\$\d*[,.]?\d+[,.]?\d+p/w',text)
                if (len(text4)==1):
                    return float(text4[0][1:-3].replace(',',''))
                elif (len(text4) > 1):
                    total = 0
                    for i in text4:
                        i = i[1:-3].replace(',','')
                        total += float(i)
                    return total/len(text4)      

In [97]:
house_df['rent'] = house_df['price'].apply(get_weekly_price)
house_df.to_csv("../data/curated/house_df_new.csv")

In [563]:
house_df_combined = house_df.reset_index().drop(['level_0'],axis = 1)

In [99]:
import ast
# get the state that houses located
state = []
for i in range(len(house_df)): 
    s = ast.literal_eval(house_df['addressParts'][i])['stateAbbreviation']
    state.append(s)
house_df['state'] = state

In [101]:
# only keep houses in VIC
house_df = house_df[house_df['state'] == 'vic'].drop('index', axis = 1)
house_df

,propertyTypes,objective,addressParts,advertiserIdentifiers,bathrooms,bedrooms,carspaces,dateUpdated,dateListed,geoLocation,...,landAreaSqm,dateAvailable,postcode,surburb,year,month,price,bond,rent,state
0,['house'],rent,"{'stateAbbreviation': 'vic', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 1...",1.0,3.0,1.0,2019-02-12T04:53:24.147Z,2014-06-04T04:40:45Z,"{'latitude': -36.3567799, 'longitude': 146.688...",...,NaN,NaN,3747,beechworth,2019,2,$320 per week,NaN,320.0,vic
1,['house'],rent,"{'stateAbbreviation': 'vic', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 1...",1.0,3.0,0.0,2020-03-27T03:59:01.72Z,2020-02-11T23:09:18Z,"{'latitude': -36.3672451, 'longitude': 146.681...",...,NaN,2020-03-20,3747,beechworth,2020,3,340.0,1473.0,340.0,vic
2,['house'],rent,"{'stateAbbreviation': 'vic', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 1...",1.0,4.0,1.0,2019-11-24T23:57:24.163Z,2019-09-16T00:14:18Z,"{'latitude': -36.3571552, 'longitude': 146.687...",...,NaN,2019-10-18,3747,beechworth,2019,11,$360 per week,NaN,360.0,vic
3,['house'],rent,"{'stateAbbreviation': 'vic', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 1...",1.0,3.0,1.0,2020-05-05T05:32:13.28Z,2020-04-14T04:46:54Z,"{'latitude': -36.3545598, 'longitude': 146.689...",...,NaN,2020-05-08,3747,beechworth,2020,5,$350 per week,NaN,350.0,vic
4,['house'],rent,"{'stateAbbreviation': 'vic', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 1...",1.0,2.0,0.0,2019-11-25T04:02:27.697Z,2019-11-13T00:16:40Z,"{'latitude': -36.3560633, 'longitude': 146.689...",...,NaN,2019-12-19,3747,beechworth,2019,11,$285 per week,NaN,285.0,vic
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8598,['house'],rent,"{'stateAbbreviation': 'vic', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 2...",2.0,4.0,4.0,2022-09-05T23:17:47.523Z,2022-08-05T03:33:29Z,"{'latitude': -37.2844387, 'longitude': 144.728...",...,NaN,2022-08-11,3435,lancefield,2022,9,under application,NaN,NaN,vic
8599,['house'],rent,"{'stateAbbreviation': 'vic', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 2...",2.0,4.0,2.0,2022-09-27T23:34:22.997Z,2022-08-21T23:50:38Z,"{'latitude': -37.2844387, 'longitude': 144.728...",...,567.91,2022-08-22,3435,lancefield,2022,9,under application,NaN,NaN,vic
8600,['house'],rent,"{'stateAbbreviation': 'vic', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 2...",1.0,2.0,0.0,2022-09-27T04:46:42.247Z,2022-09-25T23:29:15Z,"{'latitude': -37.3460627, 'longitude': 144.741...",...,NaN,2022-10-28,3434,romsey,2022,9,$440 per week,NaN,440.0,vic
8601,['house'],rent,"{'stateAbbreviation': 'vic', 'displayType': 'f...","{'advertiserType': 'agency', 'advertiserId': 2...",2.0,4.0,4.0,2022-10-03T23:30:24.74Z,2022-09-26T03:48:38Z,"{'latitude': -37.3626928, 'longitude': 144.747...",...,813.00,2022-10-03,3434,romsey,2022,10,under application,NaN,NaN,vic


In [103]:
house_df.to_csv('../data/house_combined.csv')